# Can we predict individual differences in behavior from brain activity?


Developed at Neurohackweek 2017

## Set up the environment and data

In [162]:
from HCPML_plt import clfAccHist
import os
import platform
import numpy as np
import nibabel as nib
import multiprocessing
import glob

In [180]:
# location of the data on disk
datapath = os.path.abspath('/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/')

In [181]:
# select the task and parameter estimate files to run
task = 'WM' # working memory task
pe_list = ['9','10']

Import the data:

In [183]:
sub_list=[] # subject index list
pe_bin_list=[] # list of (binarized) PEs

# add each subject to the data array
for sx, subpath in enumerate(glob.glob(datapath+'/*')):    
    print subpath
    # for each of the two contrasts
    for cx,pe in enumerate(pe_list):
        sub_list.append(os.path.basename(subpath)) # save the subject ID to a subject list
        pe_bin_list.append(cx) # save the (binarized) PE to a PE list
        
        # load the parameter estimates
        fpath = os.path.join(subpath, 'MNINonLinear/Results/tfMRI_%s/'%task,
                             'tfMRI_%s_hp200_s2_level2.feat/GrayordinatesStats/'%task,
                             'cope%s.feat/pe1.dtseries.nii'%(pe))
        pe1 = nib.load(fpath)
        
        # reshape the data into a 2-D array
        sub_pe_data = np.array(pe1.get_data().reshape(pe1.get_data().shape[4:]))
        
        # add the parameter estimate data to the overall data matrix
        if not sx and not cx:
            group_data = sub_pe_data
        else:
            group_data = np.concatenate((group_data, sub_pe_data),axis=0)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/105216
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/105620
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/105923
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/106016
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/106319
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/106521
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/106824
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/107220


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/107321
/Users/kevinsitek/Dropbox (MIT)/projects/nhw17/hcp_data_sample/107422


In [335]:
os.path.basename(subpath)

'107422'

Check the data:

In [184]:
group_data.shape

(20, 91282)

In [185]:
np.array(pe_bin_list)

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [186]:
print 'x shape: ', group_data.T.shape
print 'y shape: ', len(pe_bin_list)

x shape:  (91282, 20)
y shape:  20


## Run the classifier

In [320]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

#### Set aside a master holdout test set

In [327]:
# split the data for a master holdout test data set
def split_data(x, y):
    import numpy as np
    n_subs = len(y)/2
    rand_subs = np.random.choice(range(n_subs),n_subs,replace=False)
    
    rand_indices = []
    for i, r in enumerate(rand_subs):
        rand_indices.append(r*2)
        rand_indices.append(r*2+1)  
    Xtrain = x[:n_subs,:]
    Ytrain = y[:n_subs]
    Xtest = x[n_subs:,:]
    Ytest = y[n_subs:]
    return Xtrain, Xtest, Ytrain, Ytest, rand_indices

Xtrain, Xtest, Ytrain, Ytest, rand_indices = split_data(group_data, pe_bin_list)

In [328]:
rand_indices

[4, 5, 6, 7, 16, 17, 0, 1, 18, 19, 12, 13, 8, 9, 2, 3, 10, 11, 14, 15]

In [329]:
Xtrain.shape

(10, 91282)

#### Cross-validation

In [432]:
k_folds = 5
n_subs = len(Ytrain)/2
k_size = n_subs/k_folds

predicted_values = np.zeros(len(Ytrain))

if n_subs%k_folds:
    raise ValueError('subject number not divisible by number of folds')
else:
    for k in range(k_folds):        
        k_pe_indices = []
        k_sub_range = range(k*k_size,(k+1)*k_size)
        for kr in k_sub_range:
            print 'kr = ', kr 
            k_pe_indices.append(2*kr)
            k_pe_indices.append(2*kr+1)

        print "test indices = ", k_pe_indices
        cv_Xtest = Xtrain[k_pe_indices,:]
        cv_Ytest = [Ytrain[i] for i in k_pe_indices]

        train_pe_indices = np.delete(range(len(Ytrain)),k_pe_indices)
        print "train indices = ", train_pe_indices
        cv_Xtrain = Xtrain[train_pe_indices]
        cv_Ytrain = [Ytrain[i] for i in train_pe_indices]

        #import the model
        svc = SVC(kernel='linear')

        # fit the model on the training data
        svc.fit(cv_Xtrain, cv_Ytrain)

        # predict the labels of the test data
        prediction = svc.predict(cv_Xtest)
        print "predicted values: ", prediction
        
        predicted_values[k_pe_indices] = prediction
        
        # check the accuracy of the model's predictions
        print "model accuracy: ", svc.score(cv_Xtest, cv_Ytest)
        print ""

kr =  0
test indices =  [0, 1]
train indices =  [2 3 4 5 6 7 8 9]
predicted values:  [0 1]
model accuracy:  1.0

kr =  1
test indices =  [2, 3]
train indices =  [0 1 4 5 6 7 8 9]
predicted values:  [1 1]
model accuracy:  0.5

kr =  2
test indices =  [4, 5]
train indices =  [0 1 2 3 6 7 8 9]
predicted values:  [0 1]
model accuracy:  1.0

kr =  3
test indices =  [6, 7]
train indices =  [0 1 2 3 4 5 8 9]
predicted values:  [1 1]
model accuracy:  0.5

kr =  4
test indices =  [8, 9]
train indices =  [0 1 2 3 4 5 6 7]
predicted values:  [0 1]
model accuracy:  1.0



In [459]:
print "0-back accuracy: ", 1-np.mean(predicted_values[np.array(Ytrain)==0])
print "2-back accuracy: ", np.mean(predicted_values[np.array(Ytrain)==1])

0-back accuracy:  0.6
2-back accuracy:  1.0


#### fit the model to all training data

In [ ]:
#import the model
svc = SVC(kernel='linear')

# fit the model on the training data
svc.fit(Xtrain, Ytrain)